# Robustness Analysis

Ideas:
- sort nodes by different metrics and remove them in order.
- groups nodes in different categories, sort the categories, and remove the nodes accordingly.

Todo:
- define all the metrics to track.

Due-diligence:
- calculate the compute time before pressing run.

Possible Products:
- static charts.
- create gif for every metric.
- create real-time refreshing dashboard.

# Pipeline

- Check that set of nodes of graph has same nodes as list of nodes with habitats
- Extract unique values of habitats
- Build habitat buckets (1 key for each unique value, no problem if count 2 in one bucket and 2 in another) (build one also for food groups)
- Compute probability distribution of habitat buckets
- Build node buckets with same key as habitat buckets
- Retrieve random key according to probability distribution assigned to habitat buckets

### LOG
- All nodes with rank food_group (not species) should not be removed in the perturbations, because they are to broad categories to ever die out.

- Maybe calculate metrics modulo %.

In [13]:
def habitat_bucket_probabilites(habitat_buckets, lost_habitat=['Grassland', '']):
    """ 
    Food groups have a zero probability of being removed.
    """
    proportions = []

    foud_groups = habitat_buckets.popitem() 

    for hb in list(habitat_buckets.keys()):
        proportions.append(len([s for s in hb if  s == lost_habitat]) / len(hb))  # TODO: grep string of habitats!!!!!!!!

    n = len(proportions)
    total = sum(proportions)
    min_category_proportion = 0.05  # background extinction 
    smoothing = min_category_proportion * total / (1 - min_category_proportion * n)

    smoothed_proportions = [p + smoothing for p in proportions]
    total = sum(smoothed_proportions)
    normalized_smoothed_proportions = [p / total for p in smoothed_proportions]

    normalized_smoothed_proportions.append(0) 
    habitat_buckets['food_groups'] = foud_groups

    return normalized_smoothed_proportions

In [14]:
def habitat_buckets(nodes_df):
    habitats = pd.unique(nodes_df['habitat'])

    buckets = {}  # Changed variable name here

    for habitat in habitats:
        buckets[habitat] = []

    # mock result
    buckets = {
        ('g'): (), 
        ('g', 'other'): (),
        ('g', 'other', 'other'): (),
        ('other'): (),
        ('food_groups'): ()  # pay special attention to this one 
    }

    return buckets  # And here

def bucket_probabilities(habitat_buckets):
    normalized_smoothed_proportions = habitat_bucket_probabilites(habitat_buckets, 'g')
    return normalized_smoothed_proportions

def habitat_buckets_and_probabilities(nodes_df):
    buckets = habitat_buckets(nodes_df)  # Changed function call here
    habitat_probabilities = bucket_probabilities(buckets)  # And here

    # add nodes to habitat buckets
    for _, row in nodes_df.iterrows():
        node = row['specie']
        habitat = row['habitat']
        buckets[habitat].append(node)  # And here

    return buckets, habitat_probabilities  # And finally here

In [15]:
def choose_node(buckets, habitat_probabilities):
    # Choose a bucket
    bucket_keys = list(buckets.keys())
    chosen_bucket = np.random.choice(bucket_keys, p=habitat_probabilities)

    # Choose a node from the bucket
    nodes_in_bucket = buckets[chosen_bucket]
    chosen_node = np.random.choice(nodes_in_bucket)

    return chosen_node

In [ ]:
csv = 'node_lists/all_species_and_feeding_groups'
nodes_df = pd.DataFrame(data=csv, columns=['specie', 'habitat'])
n = len(nodes_df)

buckets, habitat_probabilities = habitat_buckets_and_probabilities(csv)

""" 

for i in range(n):
    node = choose_node(buckets, habitat_probabilities)
    node_removal(node)

    # compute metrics

"""